# INTRODUCTION
Le but de ce notebook est de relever les anomalies présentes sur les notices d'exemplaire afin de pouvoir exporter des listes à destination des aquéreurs. Le script se compose des parties suivantes :
1. Import de la table `items`
2.Traduction des colonnes pour les rendre lisibles pour les collègues qui auront le tableau sous les yeux 
3. Import des tables de valeurs autorisées
4. Une suite de requêtes sur concernant les différentes colonnes de notre dataframe nommé `items`

# IMPORT DE LA TABLE `items`

In [105]:
import pandas as pd
from datetime import datetime as dt
from datetime import date

from kiblib.utils.db import DbConn

In [106]:
db_conn = DbConn().create_engine()

In [107]:
query = """SELECT i.itemnumber, i.biblionumber, i.biblioitemnumber, i.barcode, i.dateaccessioned, i.booksellerid, i.homebranch, i.price, i.replacementprice, i.replacementpricedate, i.datelastborrowed, i.datelastseen, i.stack, i.notforloan, i.damaged, i.damaged_on, i.itemlost, i.itemlost_on, i.withdrawn, i.withdrawn_on, i.itemcallnumber, i.coded_location_qualifier, i.issues, i.renewals, i.reserves, i.restricted, i.itemnotes, i.itemnotes_nonpublic, i.holdingbranch,i.timestamp, i.location, i.permanent_location, i.onloan, i.cn_source, i.cn_sort, i.ccode, i.materials, i.uri, i.itype, i.more_subfields_xml, i.enumchron, i.copynumber, i.stocknumber, i.new_status, i.exclude_from_local_holds_priority, bi.itemtype, b.title, b.author
FROM koha_prod.items i
JOIN koha_prod.biblioitems bi ON bi.biblionumber = i.biblionumber
JOIN koha_prod.biblio b ON b.biblionumber = i.biblionumber"""

items = pd.read_sql(query,db_conn)

# MODIFICATION DU DATAFRAME & AMÉLIORATION DE LA LISIBILITÉ

In [108]:
items.columns # avant la traduction

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'barcode',
       'dateaccessioned', 'booksellerid', 'homebranch', 'price',
       'replacementprice', 'replacementpricedate', 'datelastborrowed',
       'datelastseen', 'stack', 'notforloan', 'damaged', 'damaged_on',
       'itemlost', 'itemlost_on', 'withdrawn', 'withdrawn_on',
       'itemcallnumber', 'coded_location_qualifier', 'issues', 'renewals',
       'reserves', 'restricted', 'itemnotes', 'itemnotes_nonpublic',
       'holdingbranch', 'timestamp', 'location', 'permanent_location',
       'onloan', 'cn_source', 'cn_sort', 'ccode', 'materials', 'uri', 'itype',
       'more_subfields_xml', 'enumchron', 'copynumber', 'stocknumber',
       'new_status', 'exclude_from_local_holds_priority', 'itemtype', 'title',
       'author'],
      dtype='object')

In [109]:
items = items.rename(columns={"barcode":"code-barre",
                              "dateaccessioned":"date d'acquisition",
                              "homebranch":"site propriétaire",
                              "holdingbranch":"site dépositaire",
                              "price":"prix",
                              "replacementprice":"coût de remplacement",
                              "datelastborrowed":"date dernier emprunt",
                              "datelastseen":"vu la dernière fois",
                              "damaged_on":"abimé le",
                              "itemlost_on":"perdu le",
                              "withdrawn_on":"désherbé le",
                              "itemcallnumber":"cote",
                              "issues":"nombre de prêts",
                              "timestamp":"date dernier changement",
                              "permanent_location":"localisation permanente",
                              "onloan":"retour prévu le",
                              "itemnotes":"notes publiques",
                              "itemnotes_nonpublic":"notes internes",
                              "author":"auteur",
                              "title":"titre"})

In [110]:
items.columns #après la traduction

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'désherbé le', 'cote',
       'coded_location_qualifier', 'nombre de prêts', 'renewals', 'reserves',
       'restricted', 'notes publiques', 'notes internes', 'site dépositaire',
       'date dernier changement', 'location', 'localisation permanente',
       'retour prévu le', 'cn_source', 'cn_sort', 'ccode', 'materials', 'uri',
       'itype', 'more_subfields_xml', 'enumchron', 'copynumber', 'stocknumber',
       'new_status', 'exclude_from_local_holds_priority', 'itemtype', 'titre',
       'auteur'],
      dtype='object')

# Utiliser la jointure de tables pour remplacer les valeurs autorisées

## Notforloan

In [111]:
query2 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'etat'"""

In [112]:
va_notforloan = pd.read_sql(query2, db_conn)

In [113]:
va_notforloan['authorised_value'] = va_notforloan['authorised_value'].astype(int)

In [114]:
items = items.merge(va_notforloan,left_on="notforloan",right_on="authorised_value",how="left")

In [115]:
items = items.rename(columns={"authorised_value": "valeurs autorisées", "lib": "statut"})

In [116]:
items = items.drop(columns=['valeurs autorisées'])

## Damaged

In [117]:
query3 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'damaged'"""

In [118]:
va_damaged = pd.read_sql(query3, db_conn)

In [119]:
va_damaged['authorised_value'] = va_damaged['authorised_value'].astype(int)

In [120]:
items = items.merge(va_damaged,left_on="damaged",right_on="authorised_value",how="left")

In [121]:
items = items.rename(columns={"lib": "abimé"})

In [122]:
items = items.drop(columns=["authorised_value"])

## itemlost

In [123]:
query4 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'lost'"""

In [124]:
va_itemlost = pd.read_sql(query4, db_conn)

In [125]:
va_itemlost['authorised_value'] = va_itemlost['authorised_value'].astype(int)

In [126]:
items = items.merge(va_itemlost,left_on="itemlost",right_on="authorised_value",how="left")

In [127]:
items = items.rename(columns={"lib": "perdu"})

In [128]:
items = items.drop(columns=["authorised_value"])

## Withdrawn

In [129]:
query5 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'retirecoll'"""

In [130]:
va_withdrawn = pd.read_sql(query5, db_conn)

In [131]:
va_withdrawn['authorised_value'] = va_withdrawn['authorised_value'].astype(int)

In [132]:
items = items.merge(va_withdrawn,left_on="withdrawn",right_on="authorised_value",how="left")

In [133]:
items = items.rename(columns={"lib": "désherbé"})

In [134]:
items = items.drop(columns=["authorised_value"])

## Ccode

In [135]:
query6 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'collection'"""

In [136]:
va_collection = pd.read_sql(query6, db_conn)

In [137]:
items = items.merge(va_collection,left_on="ccode",right_on="authorised_value",how="left")

In [138]:
items = items.rename(columns={"lib": "collection"})

In [139]:
items = items.drop(columns=["authorised_value"])

## location

In [140]:
query7 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'loc'"""

In [141]:
va_location = pd.read_sql(query7, db_conn)

In [142]:
items = items.merge(va_location,left_on="location",right_on="authorised_value",how="left")

In [143]:
items = items.rename(columns={"lib": "localisation"})

In [144]:
items = items.drop(columns=["authorised_value"])

## itemtype

In [145]:
query8 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'typedoc'"""

In [146]:
va_itemtype = pd.read_sql(query8, db_conn)

In [147]:
items = items.merge(va_itemtype,left_on="itemtype",right_on="authorised_value",how="left")

In [148]:
items = items.rename(columns={"lib": "type de support"})

In [149]:
items = items.drop(columns=["authorised_value"])

items.columns

* [x] notforloan / statut 
* [x] damaged / abimé
* [x] itemlost / perdu
* [x] withdrawn / retiré de la circulation
* [x] ccode / code collection
* [x] location / localisation

In [150]:
items.columns

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'désherbé le', 'cote',
       'coded_location_qualifier', 'nombre de prêts', 'renewals', 'reserves',
       'restricted', 'notes publiques', 'notes internes', 'site dépositaire',
       'date dernier changement', 'location', 'localisation permanente',
       'retour prévu le', 'cn_source', 'cn_sort', 'ccode', 'materials', 'uri',
       'itype', 'more_subfields_xml', 'enumchron', 'copynumber', 'stocknumber',
       'new_status', 'exclude_from_local_holds_priority', 'itemtype', 'titre',
       'auteur', 'statut', 'abimé', 'perdu', 'désherbé', 'collection',
       'localisation', 'type de support'],
      dtype='object')


# DÉFINITION DE QUELQUES VARIABLES IMPORTANTES

## Les colonnes à exporter

In [151]:
colonnes_to_export = ["collection",
                      "code-barre",
                      "titre",
                      "site propriétaire",
                      "localisation",
                      "statut",
                      "date dernier emprunt",
                      "notes publiques",
                     ]

In [152]:
colonnes_a_exporter = ["code-barre",
                       "titre",
                       "auteur",
                       "cote",
                       "date d'acquisition",
                       "site propriétaire",
                       "site dépositaire",
                       "prix",
                       "coût de remplacement",
                       "date dernier emprunt",
                       "vu la dernière fois",
                       "statut",
                       "abimé",
                       "abimé le",
                       "perdu",
                       "perdu le",
                       "désherbé",
                       "type de support",
                       "collection",
                       "nombre de prêts",
                       "date dernier changement",
                       "localisation",
                       "localisation permanente",
                       "notes publiques",
                       "notes internes"
                      ]

colonnes_a_exporter2 = ["auteur",
                        "titre",
                       "code-barre",
                       "date d'acquisition",
                       "site propriétaire",
                       "site dépositaire",
                       "prix",
                       "coût de remplacement",
                       "date dernier emprunt",
                       "vu la dernière fois",
                       "statut",
                       "abimé",
                       "abimé le",
                       "perdu",
                       "perdu le",
                       "retiré de la circulation",
                       "type de support",
                       "collection",
                       "cote",
                       "nombre de prêts",
                       "date dernier changement",
                       "localisation",
                       "localisation permanente",
                       "notes publiques",
                       "notes internes"
                      ]

## Transformation des colonnes au format `date`

In [153]:
items["date d'acquisition"] = pd.to_datetime(items["date d'acquisition"])
items["date dernier emprunt"] = pd.to_datetime(items["date dernier emprunt"])
items["vu la dernière fois"] = pd.to_datetime(items["vu la dernière fois"])
items["abimé le"] = pd.to_datetime(items["abimé le"])
items["perdu le"] = pd.to_datetime(items["perdu le"])
items["date dernier changement"] = pd.to_datetime(items["date dernier changement"])

## Les deadlines

In [154]:
deadline_3_mois = pd.to_datetime("today") - pd.Timedelta(3*31,unit='D')
deadline_6_mois = pd.to_datetime("today") - pd.Timedelta(365/2, unit='D')
deadline_1_an = pd.to_datetime("today") - pd.Timedelta(365, unit='D')
deadline_2_ans = pd.to_datetime("today") - pd.Timedelta(365*2, unit='D')

#Pour les perdus
deadline_1_semaine = pd.to_datetime("today") - pd.Timedelta(7,unit='D')
deadline_3_semaine = pd.to_datetime("today") - pd.Timedelta(3*7,unit='D')
deadline_5_semaine = pd.to_datetime("today") - pd.Timedelta(5*7,unit='D')

aujourdhui = pd.to_datetime("today")

In [155]:
print(f"deadline_3_mois = {deadline_3_mois}")
print(f"deadline_6_mois = {deadline_6_mois}")
print(f"deadline_1_an = {deadline_1_an}")
print(f"deadline_2_ans = {deadline_2_ans}")

deadline_3_mois = 2025-01-09 13:23:05.912132
deadline_6_mois = 2024-10-12 01:23:05.912644
deadline_1_an = 2024-04-12 13:23:05.913731
deadline_2_ans = 2023-04-13 13:23:05.914063


# Exclusion des notices du Musée

In [156]:
items = items[~items["site propriétaire"].isna()]
items = items[~items["site propriétaire"].str.match("MUS")]
items.groupby("site propriétaire").size()

site propriétaire
BUS      4283
MED    272004
dtype: int64

# Ajout d'un timestamp pour export fichiers excel

In [157]:
timestamp = date.today()
timestamp = timestamp.strftime("%Y%m%d")
timestamp

'20250412'

# LES DONNÉES INCOHÉRENTES

In [158]:
print(f"Le notebook a été executé pour la dernière fois le {aujourdhui}")

Le notebook a été executé pour la dernière fois le 2025-04-12 13:23:05.915267


In [159]:
# Incohérence site MED vs localisation
items.loc[(items['site propriétaire'] == 'MED') & 
          (items['location'].str[0:3] != 'MED') &
          (~items['location'].isna()),
         ['Localisations incohérentes - site MED']] = 'oui'


#Incohérence site BUS vs localisation
items.loc[(items["site propriétaire"] == "BUS") &
          (items["location"] != "BUS1A"),
          ['Localisations incohérentes - site BUS']] = 'oui'

In [160]:
items.columns.sort_values()

Index(['Localisations incohérentes - site BUS',
       'Localisations incohérentes - site MED', 'abimé', 'abimé le', 'auteur',
       'biblioitemnumber', 'biblionumber', 'booksellerid', 'ccode', 'cn_sort',
       'cn_source', 'code-barre', 'coded_location_qualifier', 'collection',
       'copynumber', 'cote', 'coût de remplacement', 'damaged',
       'date d'acquisition', 'date dernier changement', 'date dernier emprunt',
       'désherbé', 'désherbé le', 'enumchron',
       'exclude_from_local_holds_priority', 'itemlost', 'itemnumber',
       'itemtype', 'itype', 'localisation', 'localisation permanente',
       'location', 'materials', 'more_subfields_xml', 'new_status',
       'nombre de prêts', 'notes internes', 'notes publiques', 'notforloan',
       'perdu', 'perdu le', 'prix', 'renewals', 'replacementpricedate',
       'reserves', 'restricted', 'retour prévu le', 'site dépositaire',
       'site propriétaire', 'stack', 'statut', 'stocknumber', 'titre',
       'type de support', 

In [161]:
# Code-barres KO
items.loc[(items['code-barre'].isna()) &
          (~items['notforloan'].isin([-2,-1])),
         ['Donnée manquante - code à barre']] = "oui"

items.loc[(items['localisation'].isna()) &
          (~items['notforloan'].isin([-1,-2,4])),
         ['Donnée manquante - localisation']] = 'oui'

In [162]:
items.loc[(~items['localisation'].fillna('').str.contains('Mag')) &
          (~items['localisation'].fillna('').str.contains('Pé')) &
          (items['prix'].isna()),
         ['Donnée manquante - prix']] = 'oui'

In [163]:
items.loc[(items['collection'].isna()) &
         (~items['notforloan'].isin([-1,-2,4])),
         ['Donnée manquante - collection']] = 'oui'

In [164]:
items.loc[(items['cote'].isna()) &
         (~items['notforloan'].isin([-1,-2,4])),
         ['Donnée manquante - cote']] = 'oui'

In [165]:
items.loc[(items['type de support'].isna()) &
         (~items['notforloan'].isin([-1,-2,4])),
         ['Donnée manquante - type de support']] = 'oui'

In [166]:
items.groupby(['notforloan','statut']).size()

notforloan  statut                                 
-4          5 - En réparation                             690
-3          6 -  En retrait                              1617
-2          2 - En traitement                             951
-1          1 - En commande                                80
 0          3 - Empruntable                            210571
 2          4 - Réservé à la consultation sur place     51733
 4          8 - Sorti des collections                   10543
 5          7 - En reliure                                 93
dtype: int64

In [167]:
items.groupby('perdu le').size()

perdu le
2017-02-12 02:10:02    14
2017-02-12 02:10:03     4
2017-04-02 02:10:03     1
2017-04-09 02:10:02     1
2017-05-07 02:10:02     2
                       ..
2025-04-09 10:23:31     1
2025-04-09 10:23:57     1
2025-04-09 15:48:35     1
2025-04-11 14:56:57     1
2025-04-11 15:14:34     1
Length: 1718, dtype: int64

In [168]:
# En réparation depuis plus d'un an
items.loc[(items['abimé']=='Abimé') &
              (items['abimé le']>deadline_1_an) &
              (items['notforloan']==-4),
              ["Statut Abimé et en réparation depuis plus d'1 an"]] = 'oui'

In [182]:
# Perdus depuis 1 semaine
items.loc[(items['perdu']=='Perdu') &
          (items['perdu le']>deadline_1_semaine),
          ["Statut Perdu depuis"]] = '1 semaine'

# Perdus depuis 3 semaines
items.loc[(items['perdu']=='Perdu') &
          (items['perdu le']>deadline_3_semaine),
          ["Statut Perdu depuis"]] = '3 semaines'

#Perdus depuis 5 semaines
items.loc[(items['perdu']=='Perdu') &
          (items['perdu le']>deadline_5_semaine),
          ["Statut Perdu depuis"]] = '5 semaines'

In [176]:
# En commande pas modifié depuis 6 mois
items.loc[(items['notforloan']==-1) &
          (items['date dernier changement']>deadline_6_mois),
        ['Statut En commande depuis plus de 6 mois']] = 'oui'

In [177]:
# En traitement pas modifié depuis 6 mois
items.loc[(items['notforloan']==-2) &
          (items['date dernier changement']>deadline_6_mois),
         ['Statut En traitement depuis plus de 6 mois']] = 'oui'

In [178]:
# En retrait pas modifié depuis 6 mois
items.loc[(items['notforloan']==-3) &
          (items['date dernier changement']>deadline_6_mois),
         ['Statut En retrait depuis plus de 6 mois']] = 'oui'

In [179]:
# En sortie des collections qui devraient être supprimés de la base
items.loc[(items['notforloan']==4) &
          (items['retour prévu le'].isna()) &
          (items['date dernier changement']>deadline_6_mois),
         ['Statut Sorti des collections depuis 6 mois et pas supprimé']] = 'oui'

In [192]:
columns2export = ['code-barre',
                  'titre',
                  'Localisations incohérentes - site MED',
                  'Localisations incohérentes - site BUS',
                  'Donnée manquante - code à barre',
                  'Donnée manquante - collection',
                  'Donnée manquante - cote',
                  'Donnée manquante - localisation',
                  'Donnée manquante - prix',
                  'Donnée manquante - type de support',
                  "Statut Abimé et en réparation depuis plus d'1 an",
                  "Statut Perdu depuis",
                  "Statut En commande depuis plus de 6 mois",
                  "Statut En traitement depuis plus de 6 mois",
                  "Statut En retrait depuis plus de 6 mois",
                  "Statut Sorti des collections depuis 6 mois et pas supprimé"
                 ]
items_anomalies = items[columns2export]

In [193]:
items_anomalies[items_anomalies['Donnée manquante - collection']=='oui']

,code-barre,titre,Localisations incohérentes - site MED,Localisations incohérentes - site BUS,Donnée manquante - code à barre,Donnée manquante - collection,Donnée manquante - cote,Donnée manquante - localisation,Donnée manquante - prix,Donnée manquante - type de support,Statut Abimé et en réparation depuis plus d'1 an,Statut Perdu depuis,Statut En commande depuis plus de 6 mois,Statut En traitement depuis plus de 6 mois,Statut En retrait depuis plus de 6 mois,Statut Sorti des collections depuis 6 mois et pas supprimé
8239,None,Les Arbres de la liberté,NaN,NaN,oui,oui,oui,oui,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69659,None,Violin concertos,NaN,NaN,oui,oui,oui,oui,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89782,C2400000016,Agriculture et monde agricole,NaN,NaN,NaN,oui,NaN,NaN,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89827,C3200033825,Spirou,NaN,NaN,NaN,oui,NaN,NaN,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89828,C3200033873,Spirou,NaN,NaN,NaN,oui,NaN,NaN,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270549,C1400019949,Tombée du ciel,NaN,NaN,NaN,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272781,C2310000240,L'année de Plantu 2024,NaN,NaN,NaN,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273227,C1400021023,Les effacés,NaN,NaN,NaN,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274030,C3100012977,Punk ouvrier,NaN,NaN,NaN,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [205]:
filepath = '/home/kibini/kibini2/data/collections/anomalies/'
filename = 'anomalie_exemplaire'
file_timestamp = dt.today().strftime("%Y%m%d")

In [206]:
file_timestamp

'20250412'

In [209]:
items_anomalies.to_excel(f"{filepath}_anomalies_exemplaires_{file_timestamp}.xlsx")

---- 

### Site propriétaire vs localisation interne
Il existe 3 sites propriétaires pour la médiathèque de Roubaix : `MED`,`BUS` et `MUS`. Voici une traduction des requêtes effectuées ci-dessous : 
* `anomalies_loc_incoherente_med` : documents dont le site propriétaire est MED et dont le code de localisation ne commence pas par MED
* `anomalies_loc_incoherente_bus` : documents dont le site propriétaire est BUS et donc le code de localisation est différent de BUS.

In [55]:
anomalies_loc_incoherente_med = items[(items['site propriétaire'] == 'MED') & (items['location'].str[0:3] != 'MED')&(~items['location'].isna())]

In [56]:
anomalies_loc_incoherente_med.groupby(["site propriétaire","localisation"]).size()

Series([], dtype: int64)

In [57]:
anomalies_loc_incoherente_bus = items[(items["site propriétaire"] == "BUS") & (items["location"] != "BUS1A")]

In [58]:
anomalies_loc_incoherente_bus.groupby(["site propriétaire","localisation"]).size()

site propriétaire  localisation         
BUS                1er étage                1
                   Magasin collectivités    1
                   Rez-de-chaussée          4
dtype: int64

In [59]:
anomalies_loc_incoherente_med[colonnes_a_exporter].to_excel("../data/liste_anomalies_localisation_incoherente_med.xlsx",index=False)

In [60]:
anomalies_loc_incoherente_bus[colonnes_a_exporter].to_excel("../data/liste_anomalies_localisation_incoherente_bus.xlsx",index=False)

# LES DONNÉES MANQUANTES
### Code-barre manquant
Voici une description de la méthode employée pour la liste des anomalies de code-barre manquant : 
* J'ai sélectionné tous les documents dont le code-barre est manquant via la fonction `.isna()`
* J'ai ensuite exclu de la requête les documents *En commande* et *En traitement* 
* Ne restent dans les anomalies de code-barre que les documents sans code-barre avec un statut problématique.

In [67]:
codebarre_ko = items[items["code-barre"].isna()]

In [68]:
anomalies_docs_codebarre_ko = codebarre_ko[~codebarre_ko["notforloan"].isin([-2,-1])]

### Site propriétaire manquant

### Localisation manquante 

In [63]:
localisation_ko = items[items["localisation"].isna()]

In [64]:
anomalies_localisation_ko = localisation_ko[~localisation_ko["notforloan"].isin([-1,-2,4])]

### Prix manquant

Voici une description de la méthode employée pour générer la liste des anomalies d'exemplaires sans prix :
* J'ai défini la variable `prix_ko` comme la liste des exemplaires sans code-barre renseigné
* J'ai ensuite exclu de cette liste les documents du magasin, ainsi que les périodiques (défini par la variable `anomalies_prix_ko_coll`
* Grâce à un `groupby` et `.dt.year`, j'ai généré une liste du total de documents sans prix par an.
* On constate que c'est **à partir de 2020 que les documents sans prix augmentent**
* J'ai donc redéfini ma variable `anomalies_prix_ko_coll` comme la liste des exemplaires acheté il y a moins de 3 ans

In [65]:
prix_ko = items[items["prix"].isna()]
prix_ko["localisation"] = prix_ko["localisation"].astype(str)

/tmp/ipykernel_12936/3213706575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prix_ko["localisation"] = prix_ko["localisation"].astype(str)


In [66]:
anomalies_prix_ko_coll = prix_ko[~prix_ko["localisation"].str.contains('Mag')]

In [67]:
anomalies_prix_ko_coll = anomalies_prix_ko_coll[~anomalies_prix_ko_coll.localisation.str.contains('Pé')]

In [68]:
anomalies_prix_ko_coll = anomalies_prix_ko_coll[anomalies_prix_ko_coll["date d'acquisition"].dt.year >= 2020]

### Collection manquante

In [69]:
collection_ko = items[items["collection"].isna()]

In [70]:
anomalies_collection_ko = collection_ko[~collection_ko["notforloan"].isin([-1,-2,3,4])]

In [71]:
anomalies_collection_ko_med = anomalies_collection_ko[anomalies_collection_ko["site propriétaire"].str.match("MED")]
anomalies_collection_ko_med.groupby("site propriétaire").size()

site propriétaire
MED    203
dtype: int64

### Côte manquante

In [72]:
cote_ko = items[items["cote"].isna()]

In [73]:
anomalies_cote_ko = cote_ko[~cote_ko["notforloan"].isin([-1,-2,4])]

### Type de support manquant

In [74]:
anomalies_type_support_ko = items[items["type de support"].isna()]

In [75]:
# DES DATAFRAMES VERS UN FICHIER UNIQUE EXCEL
with pd.ExcelWriter(f"../data/collections/anomalies/anomalies_exemplaires_infos_manquantes_{timestamp}.xlsx") as writer:
    anomalies_docs_codebarre_ko[colonnes_a_exporter].to_excel(writer,sheet_name="codebarre_ko",index=False)
    localisation_ko[colonnes_a_exporter].to_excel(writer,sheet_name="localisation_ko",index=False)
    anomalies_prix_ko_coll[colonnes_a_exporter].to_excel(writer,sheet_name="prix_ko",index=False)
    anomalies_collection_ko_med[colonnes_a_exporter].to_excel(writer,sheet_name="code_collection_ko",index=False)
    anomalies_cote_ko[colonnes_a_exporter].to_excel(writer,sheet_name="cote_ko",index=False)
    anomalies_type_support_ko[colonnes_a_exporter].to_excel(writer,sheet_name="type_support_ko",index=False)

# LES PROBLEMES DE STATUT

In [61]:
vue_ensemble_statuts = items.groupby(["site propriétaire","notforloan","statut"]).size()
vue_ensemble_statuts

site propriétaire  notforloan  statut                                 
BUS                -4          5 - En réparation                               9
                   -3          6 -  En retrait                                41
                   -2          2 - En traitement                               9
                    0          3 - Empruntable                              3692
                    4          8 - Sorti des collections                     318
MED                -4          5 - En réparation                             943
                   -3          6 -  En retrait                              1313
                   -2          2 - En traitement                             741
                   -1          1 - En commande                               122
                    0          3 - Empruntable                            206570
                    2          4 - Réservé à la consultation sur place     51654
                    4          8 - Sor

### Docs Prétendus rendus statut empruntable et pas perdus

In [77]:
anomalies_docs_pretendus_rendus_mais_statut_empruntable = items[(items['abimé']=='Prétendu rendu') &
                                                          (items['statut']=='3 - Empruntable') &
                                                          (items['perdu']=='Non')
                                                         ]

# docs_pretendus_rendus_mais_statut_empruntable

### Documents abimés

Voici une description de la méthode employée pour générer la liste des anomalies d'exemplaires abimés : 
* J'ai d'abord affiché une liste des valeurs pour la colonne `abimé` (cela servira pour les requêtes)
* J'ai ensuite défini la variable `docs_abimés` comme la liste des documents marqués en `Abimé`
* J'ai ensuite défini la variable `anomalies_docs_abimes`comme la liste des docs abimés ayant été marqué en abimé depuis plus d'un an (grâce à la variable `deadline_1_an`)
* J'ai ensuit utilisé la fonction `groupby` afin d'afficher la liste des docs abimés par statut
* J'ai pu filtrer et générer deux listes différentes grâce àa la fonction `.localisation.str.contains` : 
 - la première comprends les anomalies localisées ailleurs que dans les magasins `anomalies_docs_abimes_coll`
 - La seconde liste concerne les anomalies localisées dans les différents magasins `anomalies_docs_abimes_mag` 
* **On a comme résultat une liste des documents marqués comme abimés depuis plus d'un an et qui ne sont ni sortis des collections ni des documents localisés au magasin**

In [62]:
docs_abimes = items[items["abimé"]=="Abimé"]
anomalies_docs_abimes = docs_abimes[docs_abimes["abimé le"]<deadline_1_an]
anomalies_docs_abimes.groupby("statut").size()
# Les docs abimés avec statut SORTI DES COLLECTIONS ou CONSULTABLE AU MUSEE ne sont pas des anomalies, on les exclue :
anomalies_docs_abimes = anomalies_docs_abimes[~anomalies_docs_abimes["notforloan"].isin([4,3])]
anomalies_docs_abimes["localisation"] = anomalies_docs_abimes["localisation"].astype(str)

In [78]:
anomalies_docs_abimes_coll = anomalies_docs_abimes[~anomalies_docs_abimes.localisation.str.contains('Mag')]
anomalies_docs_abimes_mag = anomalies_docs_abimes[anomalies_docs_abimes.localisation.str.contains('Magasin')]

### Documents en retrait

In [79]:
docs_en_retrait = items[items['notforloan'].isin([-3])]
anomalies_docs_en_retrait = docs_en_retrait[docs_en_retrait["date dernier changement"] < deadline_1_an]

### Documents en commande

In [80]:
docs_en_commande = items[items['notforloan'].isin([-1])]
anomalies_docs_en_commande = docs_en_commande[docs_en_commande['date dernier changement'] < deadline_6_mois]

### Documents en traitements

In [81]:
docs_en_traitement = items[items["notforloan"].isin([-2])]
anomalies_docs_en_traitement = docs_en_traitement[docs_en_traitement["date dernier changement"] < deadline_3_mois]

### Documents en réparation

In [82]:
docs_en_reparation = items[items['notforloan'].isin([-4])]
anomalies_docs_en_reparation = docs_en_reparation[docs_en_reparation["date dernier changement"] < deadline_1_an]

### Documents en reliure

In [83]:
anomalies_docs_en_reliure = items[items["notforloan"]== 5]

In [84]:
perios = items[items['type de support']  == 'Périodique']

In [86]:
with pd.ExcelWriter(f"../data/collections/anomalies/anomalies_exemplaires_prblm_statuts_{timestamp}.xlsx") as writer:
    anomalies_docs_abimes_coll[colonnes_a_exporter].to_excel(writer,sheet_name="doc_abime_coll",index=False)
    anomalies_docs_abimes_mag[colonnes_a_exporter].to_excel(writer,sheet_name="doc_abime_mag",index=False)
    anomalies_docs_en_retrait[colonnes_a_exporter].to_excel(writer,sheet_name="doc_en_retrait",index=False)
    anomalies_docs_en_commande[colonnes_a_exporter].to_excel(writer,sheet_name="doc_en_commande",index=False)
    anomalies_docs_en_traitement[colonnes_a_exporter].to_excel(writer,sheet_name="doc_en_traitement",index=False)
    anomalies_docs_en_reparation[colonnes_a_exporter].to_excel(writer,sheet_name="doc_en_reparation",index=False)
    anomalies_docs_en_reliure[colonnes_a_exporter].to_excel(writer,sheet_name="doc_en_reliure",index=False)
    anomalies_docs_pretendus_rendus_mais_statut_empruntable[colonnes_a_exporter].to_excel(writer,sheet_name='docs_pretendus_rendus',index=False)

# Nouvelles colonnes à récuperer